<a href="https://colab.research.google.com/github/mirpouya/Transformer_EDU/blob/main/einsum_and_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Think about the situation that you want to merge d dimsnsion of a 4D matrix

In [1]:
import torch
from torch import nn

In [2]:
rand_tensor = torch.rand((2, 5))
rand_tensor

tensor([[0.6676, 0.1100, 0.2785, 0.7718, 0.5302],
        [0.0416, 0.1539, 0.6134, 0.4935, 0.7339]])

In [3]:
rand_tensor = torch.rand((3,4,5,6))
rand_tensor

tensor([[[[0.8489, 0.0483, 0.5378, 0.9894, 0.8566, 0.1870],
          [0.8803, 0.8082, 0.5261, 0.5652, 0.4972, 0.6773],
          [0.7561, 0.7341, 0.8577, 0.0951, 0.7072, 0.9461],
          [0.4221, 0.3148, 0.6844, 0.0288, 0.5869, 0.3265],
          [0.3065, 0.2786, 0.1006, 0.7117, 0.0231, 0.4508]],

         [[0.1693, 0.1754, 0.7354, 0.8866, 0.1924, 0.3853],
          [0.8709, 0.8777, 0.3474, 0.8770, 0.4972, 0.2747],
          [0.1752, 0.2968, 0.5154, 0.9932, 0.3690, 0.2059],
          [0.8455, 0.4737, 0.0228, 0.3239, 0.1615, 0.1288],
          [0.4957, 0.5910, 0.8496, 0.5045, 0.4531, 0.3430]],

         [[0.3197, 0.1727, 0.3890, 0.9110, 0.7578, 0.2692],
          [0.7065, 0.7479, 0.7896, 0.9538, 0.3639, 0.0738],
          [0.7084, 0.0436, 0.2143, 0.8620, 0.3914, 0.5374],
          [0.3876, 0.7103, 0.1160, 0.5582, 0.3218, 0.0496],
          [0.9270, 0.0134, 0.5143, 0.6567, 0.3753, 0.1027]],

         [[0.3410, 0.0578, 0.9611, 0.3014, 0.6937, 0.9209],
          [0.0501, 0.8220, 0.8667,

In [4]:
!pip install einops

In [5]:
from einops import rearrange, reduce, repeat

In [6]:
rand_tensor.size()

torch.Size([3, 4, 5, 6])

In [7]:
rand_tensor = rearrange(rand_tensor, "b c h w -> (b w) c h")

# b = 3, c = 4, h = 5, w = 6
# to (3 * 6, 4, 5)

In [8]:
rand_tensor.size()

torch.Size([18, 4, 5])

### <b> einsum </b>

In [9]:
a = torch.rand((10, 10, 30))
a.size()

torch.Size([10, 10, 30])

In [10]:
a = torch.rand(10, 20, 30)
a.size()

torch.Size([10, 20, 30])

In [11]:
a = torch.randn(10, 20, 30)  # b = 10, i = 20, k = 30
b = torch.randn(10, 50, 30)  # b = 10, i = 50, k = 30

In [12]:
y = torch.einsum("b i k, b j k -> b i j", a, b)

In [13]:
y.size()

torch.Size([10, 20, 50])

In [14]:
import numpy as np

A = np.array([0, 1, 2])

B = np.array([[ 0,  1,  2,  3],
              [ 4,  5,  6,  7],
              [ 8,  9, 10, 11]])

In [15]:
np.einsum('i,ij->i', A, B)

array([ 0, 22, 76])

# <b> Transformer's blocks implementation </b>

## <b> Scaled dot product self-attention </b>

In the article, after embedding and positional embedding, embeddings are multiplied by weight matrices  <b> W_Q </b>,  <b> W_K </b>,  <b> W_V </b>.

the input <b> X </b> to the attention block is of shape: (batch, sequence_len, embedding_dim)

The matrix multiplication happens in the <b> embedding_dim </b> dimension, regardless of batch_size and sequence_len

In [16]:
# linear projection before attention block
embed_dim = 512
input = torch.randn(10, 12, 512)   # for example

qkv_weights = nn.Linear(embed_dim, embed_dim * 3, bias = False)  # we concat q, k, v into one matrix, then multiply by qkv_weights

qkv = qkv_weights(input)

q, k, v = tuple(rearrange(qkv, "b t (d k) -> k b t d", k = 3))

In [17]:
q.size(), k.size(), v.size()

(torch.Size([10, 12, 512]),
 torch.Size([10, 12, 512]),
 torch.Size([10, 12, 512]))

<b> Step 2 </b>

calculate scaled dot product, apply mask, and finally compute softmax in d last dimension.